GIBScolors_utils.py - Utility pack for mapping and accessing GIBS     <br>
                      color scheme for AMSR-2 products and variables  <br>
                      listed in <shortForProd> and <shortForVar> in   <br>
                      the gibsCfile() function                        <br>
Notes:                                                                <br>
  1. The code is programmed for the use of the .he5-to-GeoTiff        <br>
     converter that is created by the AMSR tean @ GHRC to access the  <br>
     GIBS color schemes used for certain variables                    <br>
  2. The GIBS colors then will be used to set color map in the GeoTiff<br>
     file. The function colorTable() sets the color map.              <br>
  3. The pack comes with a collection of GIBS scheme files (in .xml). <br>
     User need to specify the location of these files by setting the  <br>
     <gibsMapDir> parameter                                           <br>
                                                                      <br>
By Y. Wu of AMSR Team @ GHRC, May 06, 2021                            <br>

In [ ]:


import os,sys
import numpy as np
from glob import glob
from osgeo import gdal

#------------------------------------------------
# Note geotiff cannot set "alpha" (transparency)
# The only transparent value is NoData
#------------------------------------------------

def rmChs(ccc,chars):
    """Remov any charaters in chars from string ccc"""
    for c in chars:
        ccc=ccc.replace(c,'')
    return ccc

def toTupl(ccc,sep,convert=None):
    """convert items in a string, separated by sep, to tuple"""
    tp = ccc.split(sep)
    if(convert):
        tp = [convert(t) for t in tp]
    return tuple(tp)

def gibsCfile(gibsMaps,ptype, vname):
    """Mapping the var/parm with product to its colormap file"""
    shortForProd ={'SeaIce6':  'SI6',
                   'SeaIce12': 'SI12',
                   'SeaIce25': 'SI25',
                   'DailySnow': 'DySno',
                   '5DaySnow': '5DSno',
                   'MonthlySnow': 'MoSno',
                   'Rain': 'Rain',
                   'Ocean': 'Ocean',
                   'Land': 'Land',    }

    shortForVar={'SI_06km_NH_89H': '89H_N',
                 'SI_06km_NH_89V': '89V_N',
                 'SI_06km_SH_89H': '89H_S',
                 'SI_06km_SH_89V': '89V_S',
                 'NH_ICECON': 'CON_N',
                 'SH_ICECON': 'CON_S',
                 'SWE_NorthernDaily': 'SWE_N',
                 'SWE_SouthernDaily': 'SWE_S',
                 'ConvectivePrecip': 'CP',
                 'FrozenPrecip': 'FP',
                 'SurfacePrecip': 'SP',
                 'TotalColWaterVapor': 'WV',
                 'TotalPrecipitableWater': 'TPW',
                 'WindSpeed': 'WS',
                 'SoilMoistureSCA': 'SCASM',
                 'SoilMoistureNPD': 'NPDSM',   }

    #--- find variable on the list
    Vnm = None
    for itm in shortForVar: 
        if(itm in vname): 
            Vnm = shortForVar[itm]
            break

    #--- find the GIBS colormap file for the var with its product type
    colorFile = None
    for cmap in gibsMaps:
        if(Vnm and (Vnm in cmap) and (shortForProd[ptype] in cmap)):
            colorFile = cmap
            
    if(colorFile): print(' Use {} for {}'.format(colorFile,vname))
    
    return colorFile

def checkForRampUp(gibsColors):
    """check for color ramp (continuous in a range)
    Check for label(s) with range a - b
    while other labels are single numbers (discrete colors)
    """
    continous = '-' in  gibsColors[0]['label']
    if(not continous):
        for i, item in gibsColors.items():
            if(i != 'nodata' and i != 'unit'):
                if ('-' in item['label']): 
                    item['ramp']=item['label'].split(' - ')

    return gibsColors

def gibsScheme(gibsMapDir, ptype, vname, scaleFC):
    """Read GIBS color scheme for variable/parameter vname"""
    
    gibsXML = [os.path.basename(s) for s in glob(gibsMapDir+'*')]
    
    #--- get GIBS colormap file
    colorFile = gibsCfile(gibsXML, ptype, vname)
    if not colorFile:
        print(' {} has no GIBS color scheme.'.format(vname))
        print(' Thus, not setting color scheme for it.\n')
        return None
    

    #--- get GIBS colors {} from the colormap file"""
    f=open(os.path.join(gibsMapDir, colorFile), 'r')
    lines = f.readlines()
    f.close()

    gibsColors, lev = {}, 0
    for line in lines: 
        
        if('units' in line):
            unit = rmChs(line.split('=')[-1],'&#xB3;">').strip('\n')
            gibsColors['unit'] = unit

        if('ColorMapEntry' in line):
            #--for 'color'
            rgb = line.split('rgb=')[1].split(' ')[0]
            rgb = toTupl(rmChs(rgb,'"'),',',int)
            
            if('nodata' in line or 'Fill' in line):
                val = 'nodata'
                gibsColors[val] = {'color': (220,220,220)} #rgb}
            else:
                #--for 'label'
                if('value=' in line):
                    aaa = line.split('value=')[-1].split(' ')[0]
                    aaa = rmChs(aaa,'"[]()')
                    lab = aaa
                else:
                    aaa = line.split('Value=')[-1].split(' ')[0]
                    aaa = rmChs(aaa,'"[]()')
                    lab = aaa
                    
                #--for 'value'
                if('sourceValue=' in line):
                    aaa = line.split('sourceValue=')[-1].split(' ')[0]
                    aaa = rmChs(aaa,'"[]()')
                    val = aaa.split(',')[0]
                    if(lab[0]=='-'): lab=val  #<--Rain_WV's value is invalid 
                else:
                    val = lab.split(',')[0]
                
                if('.' in val): val = float(val)
                else:
                    val = int(val)
                
                #---gibsColors{}
                lab = lab.replace(',',' - ')
                gibsColors[lev]={'color':rgb,
                                 'value':val,
                                 'label':lab}
                lev += 1
    
    if('SWE' in vname or 'CON' in vname):
        reScale = False
        notVal = 255

        print(' Made {} color intervals for the range [{}-{}]'.format(
        lev,gibsColors[0]['value'],gibsColors[lev-1]['value'] ))

    else:
        oldRange=[gibsColors[0]['value'], gibsColors[lev-1]['value']]
        dV = (oldRange[1]-oldRange[0])/lev
        scaleFC['old']=[oldRange[0],oldRange[1], None]
        scaleFC['new']=[0, lev-1, None]

        print(' Made {} color intervals for the range [{}-{}]'.format(
        lev,gibsColors[0]['value'],gibsColors[lev-1]['value'] ))

    gibsColors = checkForRampUp(gibsColors)
    
    return gibsColors


def colorTable(gibsColors, nodata, scaleFC, verb=False):
    
    colors = gdal.ColorTable()
    
    unit='-'
    for lev, item in gibsColors.items():
        if(lev == 'unit'):
            unit = item
        elif(lev == 'nodata'):
            #print(nodata,type(nodata), item['color'], type(item['color']))
            colors.SetColorEntry(int(nodata), item['color'])
        else:
            if('ramp' in item):
                lev1, lev2 = int(item['ramp'][0]), int(item['ramp'][1])
                colors.CreateColorRamp(lev1, item['color'], lev2, item['color'])
            else:
                if(scaleFC):lev1 = lev
                else:       lev1 = item['value']
                colors.SetColorEntry(lev1, item['color'])
                if(verb): print(' Value {} corresponds to {} ({})'.format(
                    lev1,item['label'],unit))

    return colors